In [2]:
import ffmpeg

def create_normalized_temp_file(file_path: str) -> None:
    '''
    Created a video with normalized fps (30), codec (h264),
    video timescale (90000), video profile (-profile:v),
    and resolution (480p). file_path has a 'temp" prefix.
    '''

    stream = ffmpeg.input(file_path)

    stream = ffmpeg.output(stream, 'temp_' + file_path, 
        **{
            'r': 30,  # fps
            'vcodec': 'libx264', #codec
            's': '640x480',  # resolution
            'video_track_timescale': 90000, #timescale
            'profile:v': 'baseline',
        }
    )
    
    try:
        stream.overwrite_output().run(capture_stdout = True, capture_stderr = True)
    except ffmpeg.Error as e:
        print('stdout:', e.stdout.decode('utf8'))
        print('stderr:', e.stderr.decode('utf8'))

In [3]:
#create_normalized_temp_file('clipX.mp4')
#create_normalized_temp_file('clipY.mp4')

In [3]:
def create_segment(file_path, start_time, duration):
    """
    Create a fresh video segment from input file
    """
    return (
        ffmpeg
        .input(file_path, ss = start_time)  # Use ss parameter for more accurate seeking
        .video
        .filter('trim', duration = duration)
        .filter('setpts', 'PTS-STARTPTS')  # Simplify PTS adjustment
        .filter('fps', fps = 30)
        .filter('minterpolate', mi_mode = 'dup', fps = 30)  # Match output fps
    )

In [ ]:
# Alternate between 10-second segments of in1 and in2
segments = []
durations = [10, 11, 11, 11, 11, 11]

for i, d in zip(range(0, 60, 10), durations):

    current_file = 'clipX.mp4' if i % 20 == 0 else 'clipY.mp4'

    segments.append(
        create_segment(
        file_path = current_file,
        start_time = i,
        duration = d
        )
    )

# Create cross-dissolve
current = segments[0]
for i in range(1, len(segments)):

    current = ffmpeg.filter(
        [current, segments[i]],
        'xfade',
        transition = 'fade',
        duration = 1,
        offset = 9 + (10 * (i - 1))
        )

# Add subtitles with custom formatting
current = current.filter(
        'subtitles',
        filename = 'puppy_subs2.srt',
        force_style = 'Fontsize=30,FontName=Comic Sans MS,PrimaryColour=&HFFFFFF,OutlineColour=&H000000,BorderStyle=3,Outline=2,Shadow=1,MarginV=30'
    )

# Add brightness adjustment to final frames and freeze frame with text
current = (
    current
    .filter('tpad', stop_mode = 'clone', stop_duration = 4)  # Add 4 seconds of frozen last frame
    .filter('curves', preset = 'lighter', enable = f'between(t,{56},999)')  # Lighten final 8 seconds
    .filter('drawtext',
            text = 'Puppy Enough.',
            fontcolor = 'white',
            fontsize = 60,
            font = 'Comic Sans MS',
            x = '(w-text_w)/2',  # Center horizontally
            y = '(h-text_h)/2',  # Center vertically
            enable = f'between(t,{60},999)',  # Show text in final 4 seconds
            shadowcolor = 'black',
            shadowx = 2,
            shadowy = 2
            )
)

# Bark sound overlay
bark_audio = (
    ffmpeg.input('tiny_bark.wav')
    .filter('volume', '0.6')  # 60% volume
    .filter('aformat', sample_fmts = 'fltp', sample_rates = 44100, channel_layouts = 'mono')
    .filter('adelay', '40000')  # Delay by 40000ms (40s) - only one channel now
    .filter('apad', pad_dur = 40)  
    .filter('atrim', duration = 42)  
)

# Write output
output = ffmpeg.output(
    current,
    bark_audio,
    'output.mp4',
    **{
        # Video settings
        'vcodec': 'libx264',
        'r': 30,
        's': '640x480',
        'video_track_timescale': 90000,
        'profile:v': 'baseline',
        'preset': 'medium',
        'pix_fmt': 'yuv420p',  # Standard pixel format for compatibility
        'vsync': 'cfr',
        
        # Audio settings
        'acodec': 'aac',
        'ac': 1,              # Mono
        'ar': 44100,          # Standard sample rate
        'ab': '128k',         # Standard bitrate
    }
)
compiled = ffmpeg.compile(output)
try:
    output.overwrite_output().run(capture_stdout = True, capture_stderr = True)
except ffmpeg.Error as e:
    print('stdout:', e.stdout.decode('utf8'))
    print('stderr:', e.stderr.decode('utf8'))
compiled


In [52]:
def create_segment2(file_path):
    """
    Create a fresh video segment from input file
    """
    return (
        ffmpeg
        .input(file_path)  # Use ss parameter for more accurate seeking
        .video
        .filter('setpts', 'PTS-STARTPTS')  # Simplify PTS adjustment
        .filter('fps', fps = 30)
        .filter('scale', 'iw/2', 'ih/2')
        #.filter('minterpolate', mi_mode = 'dup', fps = 30)  # Match output fps
    )

In [ ]:
import ffmpeg

# Input video
# Scales down by 50%, Sets fps to 30, Trim to 60s

inp_front = create_segment2('multicam_wedding_front.mov')
inp_side = create_segment2('multicam_wedding_side.mov')

# Horizontal Stacking

stream2 = (
    ffmpeg
    .filter([inp_front, inp_side], 'hstack')
    .split()
)

# Freeze at 30 seconds in for 2 seconds

before_s2 = (
    stream2['0']
    .filter('trim', start = 0, end = 30)
    .filter('setpts', 'PTS-STARTPTS')
)

freeze_s2 = (
    stream2['1']
    .filter('trim', start = 30, duration = 0.033)
    .filter('tpad', stop_mode = 'clone', stop_duration = 2)
    .filter('setpts', 'PTS-STARTPTS')
)

after_s2 = (
    stream2['2']
    .filter('trim', start = 30)
    .filter('setpts', 'PTS-STARTPTS')
)

stream2 = ffmpeg.concat(
    before_s2,
    freeze_s2,
    after_s2,
    v = 1  
)

# Underlay soft_strings3.mp3 at 30% from halfway onwards
d1 = ffmpeg.probe('multicam_wedding_front.mov')["format"]["duration"]
d2 = ffmpeg.probe('multicam_wedding_side.mov')["format"]["duration"]
L = max(float(d1), float(d2))
delay = (L + 2) / 2

audio_overlay = (
    ffmpeg
    .input('soft_strings3.mp3')
    .filter('volume', 0.3)  # 30% volume
    .filter('aformat', channel_layouts = 'mono')
    .filter('adelay', f'{1000 * delay}') 
    .filter('atrim', duration = 2 * delay)  # Trim to match video length
)

# Add saturation to final 5 seconds

stream2 = stream2.filter('eq', saturation = 1.3, enable = f'between(t,{L - 5},999)')

# Swirl effect at the end of the video

stream2 = (
    stream2
    .filter(
        'rotate',
        angle = f'if(gte(t,{L}),(t-{L})*360,0)',
        fillcolor = 'black',
        enable = f'gte(t,{L})')
    .filter(
        'fade',
        type='out',
        start_time = L,
        duration = 2)
)

# Output video

out2 = ffmpeg.output(
    stream2,
    audio_overlay,
    'output2.mp4',
    **{
        # Video settings
        'vcodec': 'libx264',
        'r': 30,
        'video_track_timescale': 90000,
        'preset': 'medium',
        'pix_fmt': 'yuv420p',  # Standard pixel format for compatibility
        
        # Audio settings
        'acodec': 'aac',
        'ac': 1,              # Mono
        'ar': 44100,          # Standard sample rate
        'ab': '128k',         # Standard bitrate
    }
)
compiled2 = ffmpeg.compile(out2)
try:
    out2.overwrite_output().run(capture_stdout = True, capture_stderr = True)
except ffmpeg.Error as e:
    print('stdout:', e.stdout.decode('utf8'))
    print('stderr:', e.stderr.decode('utf8'))
compiled2

In [1]:
def create_segment3(file_path, start_time, duration):
    """
    Create a fresh video segment from input file
    """
    return (
        ffmpeg
        .input(file_path, ss = start_time)  # Use ss parameter for more accurate seeking
        .video
        .filter('trim', duration = duration)
        .filter('setpts', 'PTS-STARTPTS')  # Simplify PTS adjustment
        .filter('fps', fps = 30)
    )

In [ ]:
import ffmpeg

# Input video
# Scales down by 50%, Sets fps to 30, Trim to 60s

inp_noon = create_segment3('dance_recital_noon.mp4', 0, 60)
inp_even = create_segment3('dance_recital_evening.mp4', 0, 60)

# Fade-out first video and fade-in second video
# then concat them both

inp_noon = inp_noon.filter('fade', type = 'out', duration = 2, start_time = 58)
inp_even = inp_even.filter('fade', type = 'in', duration = 2, start_time = 0)
stream3 = ffmpeg.concat(inp_noon, inp_even, v = 1)

# Insert small overlay of 'Recital!'
stream3 = stream3.filter(
    'drawtext',
    text = 'Recital!',
    fontsize = 24,
    fontcolor = 'white',
    fontfile = 'Arial',  # Make sure this font is available on your system
    x = '(w-text_w)/2',  # Center horizontally
    y = '(h-text_h)/2',  # Center vertically
    enable = f'between(t,8,12)',  # Only show from 8-12 seconds
    shadowcolor = 'black',
    shadowx = 2,
    shadowy = 2
)

# Overlay dance_clapfx.wav at 50% for 1 seconds twice

clap = ffmpeg.input('dance_clapfx.wav').asplit()

clap1 = (
    clap['0']
    .filter('volume', 0.5)  # 50% volume
    .filter('aformat', channel_layouts = 'mono')
    .filter('adelay', '3000')  # Delay by 3000ms (3 seconds)
    .filter('atrim', duration = 4)  # Duration of 1 second
)

clap2 = (
    clap['1']
    .filter('volume', 0.5)  # 50% volume
    .filter('aformat', channel_layouts = 'mono')
    .filter('adelay', '3000')  # Delay by 3000ms (3 seconds)
    .filter('atrim', duration = 4)  # Duration of 1 second
)

mixed_audio = (
    ffmpeg
    .concat(clap1, clap2, a = 1, v = 0)
)

# Freeze final frames with text

stream3 = (
    stream3
    .filter('tpad', stop_mode = 'clone', stop_duration = 4)  # Add 4 seconds of frozen last frame
    .filter('drawtext',
            text = 'Finale.',
            fontcolor = 'white',
            fontsize = 60,
            font = 'Comic Sans MS',
            x = '(w-text_w)/2',  # Center horizontally
            y = '(h-text_h)/2',  # Center vertically
            enable = f'between(t,{120},999)',  # Show text in final 4 seconds
            shadowcolor = 'black',
            shadowx = 2,
            shadowy = 2
            )
)

# Output video

out3 = ffmpeg.output(
    stream3,
    mixed_audio,
    'output3.mp4',
    **{
        # Video settings
        'vcodec': 'libx264',
        'r': 30,
        'video_track_timescale': 90000,
        'preset': 'medium',
        'pix_fmt': 'yuv420p',  # Standard pixel format for compatibility
        
        # Audio settings
        'acodec': 'aac',
        'ac': 1,              # Mono
        'ar': 44100,          # Standard sample rate
        'ab': '128k',         # Standard bitrate
    }
)
compiled3 = ffmpeg.compile(out3)
try:
    out3.overwrite_output().run(capture_stdout = True, capture_stderr = True)
except ffmpeg.Error as e:
    print('stdout:', e.stdout.decode('utf8'))
    print('stderr:', e.stderr.decode('utf8'))
compiled3

['ffmpeg',
 '-ss',
 '0',
 '-i',
 'dance_recital_noon.mp4',
 '-ss',
 '0',
 '-i',
 'dance_recital_evening.mp4',
 '-i',
 'dance_clapfx.wav',
 '-filter_complex',
 '[0:v]trim=duration=60[s0];[s0]setpts=PTS-STARTPTS[s1];[s1]fps=fps=30[s2];[s2]fade=duration=2:start_time=58:type=out[s3];[1:v]trim=duration=60[s4];[s4]setpts=PTS-STARTPTS[s5];[s5]fps=fps=30[s6];[s6]fade=duration=2:start_time=0:type=in[s7];[s3][s7]concat=n=2:v=1[s8];[s8]drawtext=enable=between(t\\,8\\,12):fontcolor=white:fontfile=Arial:fontsize=24:shadowcolor=black:shadowx=2:shadowy=2:text=Recital!:x=(w-text_w)/2:y=(h-text_h)/2[s9];[s9]tpad=stop_duration=4:stop_mode=clone[s10];[s10]drawtext=enable=between(t\\,120\\,999):font=Comic Sans MS:fontcolor=white:fontsize=60:shadowcolor=black:shadowx=2:shadowy=2:text=Finale.:x=(w-text_w)/2:y=(h-text_h)/2[s11];[2]asplit=2[s12][s13];[s12]volume=0.5[s14];[s14]aformat=channel_layouts=mono[s15];[s15]adelay=3000[s16];[s16]atrim=duration=5[s17];[s13]volume=0.5[s18];[s18]aformat=channel_layouts=mo